## (If you have a reference: Benchmark the output)

For this to make any sense, you need to have a *reference*, that is, a list of bins that are deemed true and complete.

The reference could be a {clustername: set(contigs)} dict along with a {contigname: length} dict, just like the `clusters` and `lengthof` we made. It could also be a tab-separated file with (clustername, contigname, length)-rows, one row per contig.

Now, I have no reference for this dataset, so I created a reference file completely randomly:

In [ ]:
!head /home/jakni/Downloads/example/reference.tsv

---
We of course expect the benchmark to show we have at most a handful of very incomplete bins, since the reference is random.

---

In [ ]:
reference_path = '/home/jakni/Downloads/example/reference.tsv'

with open(reference_path) as filehandle:
    reference = vamb.benchmark.Reference.fromfile(filehandle)

---
We also need to instantiate the Observed bins (which we created above!), and a BenchMarkResult

---

In [ ]:
# We could also do this from the bins.tsv we created in the previous section,
# but here we have the dictionary with bins in memory already.
observed = vamb.benchmark.Observed(filtered_bins, reference)

# Keyword-only arguments to make sure you don't accidentally swap them around.
# It'll raise an error if you use non-keyword arguments.
result = vamb.benchmark.BenchMarkResult(reference=reference, observed=observed)

result = vamb.benchmark.BenchMarkResult(reference=reference, observed=observed)

# Okay, how did we do?
result.printmatrix()

---
This matrix shows the number of reference bins where, for each value of recall and precision there is at least one observed bin that passes those criteria.

As expected (because the reference was randomly generated), the results are terrible - in fact, they couldn't be worse.

To check what else the BenchMarkResult measures, check `help(vamb.benchmark.BenchMarkResult)`

---